In [1]:
from queue import Queue
import threading
from socketIO_client import SocketIO, BaseNamespace
import numpy as np
import time

In [2]:
def run_parallel_in_threads(target, args_list):
    globalparas = []
    result = Queue()
    def task_wrapper(*args):
        result.put(target(*args))
    threads = [threading.Thread(target = task_wrapper, args = args) for args in args_list]
    for t in threads:
        t.start()
    for t in threads:
        t.join()
    while not result.empty():
        globalparas.append(result.get())
    globalparas = list(filter(None, globalparas))
    return globalparas

def get_time(i):
    last_time = time.time()
    socketIO = SocketIO('http://localhost', 5000)
    live_namespace = socketIO.define(BaseNamespace, '/live')
    live_namespace.emit('event',{'id':int(i)})
    return [time.time()-last_time, i]

In [3]:
def stress_test(count, per_second):
    aranged = np.arange(count)
    for index in range(0,aranged.shape[0],per_second):
        batch = aranged[index:min(index + per_second,aranged.shape[0])]
        threads = []
        for k in batch:
            threads.append((k,))
        outputs = run_parallel_in_threads(get_time, threads)
        total = 0
        for i in outputs:
            total += float(i[0])
        print('index %d, total time taken %f s, average time taken %f s'%(index, total, total / len(outputs)))
        time.sleep(1)

In [4]:
# gunicorn with eventlet
stress_test(400,100)

index 0, total time taken 99.869447 s, average time taken 0.998694 s
index 100, total time taken 222.226329 s, average time taken 2.222263 s
index 200, total time taken 271.741829 s, average time taken 2.717418 s
index 300, total time taken 376.807925 s, average time taken 3.768079 s
